In [5]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as pre
import requests
import datetime
import numpy as np
import math
import json
import time

In [6]:
oauth_file = '../auth/oauth.txt'
keys = []
line = 'a'
with open(oauth_file) as fp:
    while line:
        line = fp.readline().strip()
        keys.append(line)

#Twitter credentials for the app
consumer_key = keys[2]
consumer_secret = keys[3]
access_key = keys[0]
access_secret = keys[1]

In [7]:
# Credentials
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [28]:
# StreamListener class inherits from tweepy.StreamListener and overrides on_status/on_error methods.
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        
        print(status.id_str)
        
        # class code adapted from https://www.storybench.org/how-to-collect-tweets-from-the-twitter-streaming-api-using-python/
        
        '''
        # if "retweeted_status" attribute exists, flag this tweet as a retweet.
        is_retweet = hasattr(status, "retweeted_status")

        # check if text has been truncated
        if hasattr(status,"extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text

        
        # check if this is a quote tweet.
        is_quote = hasattr(status, "quoted_status")
        quoted_text = ""
        if is_quote:
            # check if quoted tweet's text has been truncated before recording it
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet["full_text"]
            else:
                quoted_text = status.quoted_status.text

        # remove characters that might cause problems with csv encoding
        remove_characters = [",","\n"]
        for c in remove_characters:
            text.replace(c," ")
            quoted_text.replace(c, " ")
        '''
            
        OP_DIR = '../data/stream/'
        op_file = OP_DIR + 'tweets_' + str(status.id_str) + '.stream'

        with open(op_file, 'w', encoding='utf-8') as fp:
            result = json.dumps(status._json)
            fp.write(result + '\n')            

    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()

In [ ]:
# initialize stream
streamListener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')
with open("../data/out.csv", "w", encoding='utf-8') as f:
    f.write("date,user,is_retweet,is_quote,text,quoted_text\n")
tags = ['corona', 'covid', 'virus', 'flu']
stream.filter(track=tags, languages=['en'])